In [16]:
!pip install pandas

In [31]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 10.1 MB/s eta 0:00:0000:0100:01


In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("../data/raw/BC_A&A_with_ATD.csv")
df.shape

(1000000, 15)

# Dropping null rows

In [19]:
df_cleaned = df.replace('\\N', pd.NA).dropna()
df_cleaned.shape

(984241, 15)

# Changing data types

In [20]:
import pandas as pd
import numpy as np

df_converted = df_cleaned.copy()

# Convert "\N" strings to actual NaN values
df_converted.replace(r'\\N', np.nan, regex=True, inplace=True)

# Convert timestamp columns to datetime
timestamp_cols = [
    "restaurant_offered_timestamp_utc",
    "order_final_state_timestamp_local",
    "eater_request_timestamp_local"
]
for col in timestamp_cols:
    df_converted[col] = pd.to_datetime(df_converted[col], errors='coerce')  # Invalid formats become NaT

# Convert distance columns to float
distance_cols = [
    "pickup_distance",
    "dropoff_distance"
]
for col in distance_cols:
    df_converted[col] = pd.to_numeric(df_converted[col], errors='coerce')  # Invalid formats become NaN

In [21]:
df_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 984241 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   region                             984241 non-null  object        
 1   territory                          984241 non-null  object        
 2   country_name                       984241 non-null  object        
 3   workflow_uuid                      984241 non-null  object        
 4   driver_uuid                        984241 non-null  object        
 5   delivery_trip_uuid                 984241 non-null  object        
 6   courier_flow                       984241 non-null  object        
 7   restaurant_offered_timestamp_utc   984241 non-null  datetime64[ns]
 8   order_final_state_timestamp_local  984241 non-null  datetime64[ns]
 9   eater_request_timestamp_local      984241 non-null  datetime64[ns]
 10  geo_archetype            

# Converting dates to local time

In [22]:
from datetime import datetime
from zoneinfo import ZoneInfo  # Available in Python 3.9+

# Example: assuming the column is datetime and named `restaurant_offered_timestamp_utc`
df_converted['restaurant_offered_timestamp_local'] = df_converted['restaurant_offered_timestamp_utc'].dt.tz_localize('UTC').dt.tz_convert('America/Mexico_City')

In [23]:
df_converted["restaurant_offered_timestamp_local"] = pd.to_datetime(df_converted["restaurant_offered_timestamp_local"]).dt.tz_localize(None)

In [24]:
df_converted.head()

,region,territory,country_name,workflow_uuid,driver_uuid,delivery_trip_uuid,courier_flow,restaurant_offered_timestamp_utc,order_final_state_timestamp_local,eater_request_timestamp_local,geo_archetype,merchant_surface,pickup_distance,dropoff_distance,ATD,restaurant_offered_timestamp_local
0,Mexico,South East,Mexico,8c393c94-9282-41a6-a885-7a6e84b470d7,d16e401c-795d-4295-96c0-85ca08ad8c42,715f96aa-0a31-46f6-b856-6ea6f87affad,Motorbike,2025-04-06 19:35:50,2025-04-06 13:48:48,2025-04-06 13:35:50,Drive momentum,Tablet,0.643,1.651,12.966667,2025-04-06 13:35:50
1,Mexico,South East,Mexico,b156dd16-274f-46d1-ac6f-4c24ca43df69,06299354-23ec-4173-83dc-d6bd32a420e1,ac985454-0e6f-4695-a68b-755c60c2afd7,Motorbike,2025-04-11 22:30:56,2025-04-11 16:47:48,2025-04-11 16:30:56,Drive momentum,Tablet,0.812,1.733,16.866667,2025-04-11 16:30:56
2,Mexico,South East,Mexico,87a8081b-b735-4daf-8452-3943a9d9ca24,188214a6-3813-44d5-9694-0ae69dfdd7e5,14b274b9-924e-4da7-8803-69558a33559d,Motorbike,2025-04-06 23:15:05,2025-04-06 17:34:49,2025-04-06 17:15:05,Drive momentum,Tablet,1.966,1.605,19.733333,2025-04-06 17:15:05
3,Mexico,South East,Mexico,78f0e92c-fa45-4215-8c3f-703b8691b6eb,5c7cbb34-116b-4665-aaf3-7f02394c0ff8,ee4faf69-bfff-421a-a163-29dfa4156fc4,Motorbike,2025-04-19 02:38:58,2025-04-18 21:21:55,2025-04-18 20:38:58,Drive momentum,Tablet,3.010,4.771,42.950000,2025-04-18 20:38:58
4,Mexico,Central,Mexico,e0b2324b-f210-4e6b-82d1-979f9a208967,8b7a3bd8-9cf1-493e-9c64-303db8781045,7870b275-4541-4e09-8fe7-a47e7b506ce4,Motorbike,2025-03-03 03:33:40,2025-03-02 22:02:36,2025-03-02 21:33:40,Play offense,Tablet,0.746,4.370,28.933333,2025-03-02 21:33:40


In [25]:
df_converted = df_converted.drop('restaurant_offered_timestamp_utc', axis = 1)

In [26]:
df_converted.shape

(984241, 15)

In [27]:
df_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 984241 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   region                              984241 non-null  object        
 1   territory                           984241 non-null  object        
 2   country_name                        984241 non-null  object        
 3   workflow_uuid                       984241 non-null  object        
 4   driver_uuid                         984241 non-null  object        
 5   delivery_trip_uuid                  984241 non-null  object        
 6   courier_flow                        984241 non-null  object        
 7   order_final_state_timestamp_local   984241 non-null  datetime64[ns]
 8   eater_request_timestamp_local       984241 non-null  datetime64[ns]
 9   geo_archetype                       984241 non-null  object        
 10  merchant_surf

In [28]:
# Saving interim processed data

df_converted.to_csv('../data/interim/BC_A&A_with_ATD_interim.csv', index = False)

In [32]:
df_converted.to_parquet('../data/interim/BC_A&A_with_ATD_interim.parquet', index = False)